In [1]:
# read point cloud bin file
import numpy as np
import struct
import collections

def read_next_bytes(fid, num_bytes, format_char_sequence, endian_character="<"):
    """Read and unpack the next bytes from a binary file.
    :param fid:
    :param num_bytes: Sum of combination of {2, 4, 8}, e.g. 2, 6, 16, 30, etc.
    :param format_char_sequence: List of {c, e, f, d, h, H, i, I, l, L, q, Q}.
    :param endian_character: Any of {@, =, <, >, !}
    :return: Tuple of read and unpacked values.
    """
    data = fid.read(num_bytes)
    return struct.unpack(endian_character + format_char_sequence, data)


def read_points3D_binary(path_to_model_file):
    """
    see: src/base/reconstruction.cc
        void Reconstruction::ReadPoints3DBinary(const std::string& path)
        void Reconstruction::WritePoints3DBinary(const std::string& path)
    """


    with open(path_to_model_file, "rb") as fid:
        num_points = read_next_bytes(fid, 8, "Q")[0]

        xyzs = np.empty((num_points, 3))
        rgbs = np.empty((num_points, 3))
        errors = np.empty((num_points, 1))
        
        for p_id in range(num_points):
            binary_point_line_properties = read_next_bytes(
                fid, num_bytes=43, format_char_sequence="QdddBBBd")
            xyz = np.array(binary_point_line_properties[1:4])
            rgb = np.array(binary_point_line_properties[4:7])
            error = np.array(binary_point_line_properties[7])
            track_length = read_next_bytes(
                fid, num_bytes=8, format_char_sequence="Q")[0]
            track_elems = read_next_bytes(
                fid, num_bytes=8*track_length,
                format_char_sequence="ii"*track_length)
            xyzs[p_id] = xyz
            rgbs[p_id] = rgb
            errors[p_id] = error
    return xyzs, rgbs, errors


In [2]:
# camera / point cloud attributes

CameraModel = collections.namedtuple(
    "CameraModel", ["model_id", "model_name", "num_params"])
Camera = collections.namedtuple(
    "Camera", ["id", "model", "width", "height", "params"])
BaseImage = collections.namedtuple(
    "Image", ["id", "qvec", "tvec", "camera_id", "name", "xys", "point3D_ids"])
Point3D = collections.namedtuple(
    "Point3D", ["id", "xyz", "rgb", "error", "image_ids", "point2D_idxs"])
CAMERA_MODELS = {
    CameraModel(model_id=0, model_name="SIMPLE_PINHOLE", num_params=3),
    CameraModel(model_id=1, model_name="PINHOLE", num_params=4),
    CameraModel(model_id=2, model_name="SIMPLE_RADIAL", num_params=4),
    CameraModel(model_id=3, model_name="RADIAL", num_params=5),
    CameraModel(model_id=4, model_name="OPENCV", num_params=8),
    CameraModel(model_id=5, model_name="OPENCV_FISHEYE", num_params=8),
    CameraModel(model_id=6, model_name="FULL_OPENCV", num_params=12),
    CameraModel(model_id=7, model_name="FOV", num_params=5),
    CameraModel(model_id=8, model_name="SIMPLE_RADIAL_FISHEYE", num_params=4),
    CameraModel(model_id=9, model_name="RADIAL_FISHEYE", num_params=5),
    CameraModel(model_id=10, model_name="THIN_PRISM_FISHEYE", num_params=12)
}
CAMERA_MODEL_IDS = dict([(camera_model.model_id, camera_model)
                         for camera_model in CAMERA_MODELS])
CAMERA_MODEL_NAMES = dict([(camera_model.model_name, camera_model)
                           for camera_model in CAMERA_MODELS])

In [3]:
# Quarternion vector (4D) to Euler rotation matrix (3x3)

def qvec2rotmat(qvec):
    return np.array([
        [1 - 2 * qvec[2]**2 - 2 * qvec[3]**2,
         2 * qvec[1] * qvec[2] - 2 * qvec[0] * qvec[3],
         2 * qvec[3] * qvec[1] + 2 * qvec[0] * qvec[2]],
        [2 * qvec[1] * qvec[2] + 2 * qvec[0] * qvec[3],
         1 - 2 * qvec[1]**2 - 2 * qvec[3]**2,
         2 * qvec[2] * qvec[3] - 2 * qvec[0] * qvec[1]],
        [2 * qvec[3] * qvec[1] - 2 * qvec[0] * qvec[2],
         2 * qvec[2] * qvec[3] + 2 * qvec[0] * qvec[1],
         1 - 2 * qvec[1]**2 - 2 * qvec[2]**2]])

# Image_ class

class Image_(BaseImage):
    def qvec2rotmat(self):
        return qvec2rotmat(self.qvec)

In [4]:
# read camera binary files

def read_extrinsics_binary(path_to_model_file):
    """
    see: src/base/reconstruction.cc
        void Reconstruction::ReadImagesBinary(const std::string& path)
        void Reconstruction::WriteImagesBinary(const std::string& path)
    """
    images = {}
    with open(path_to_model_file, "rb") as fid:
        num_reg_images = read_next_bytes(fid, 8, "Q")[0]
        for _ in range(num_reg_images):
            binary_image_properties = read_next_bytes(
                fid, num_bytes=64, format_char_sequence="idddddddi")
            image_id = binary_image_properties[0]
            qvec = np.array(binary_image_properties[1:5])
            tvec = np.array(binary_image_properties[5:8])
            camera_id = binary_image_properties[8]
            image_name = ""
            current_char = read_next_bytes(fid, 1, "c")[0]
            while current_char != b"\x00":   # look for the ASCII 0 entry
                image_name += current_char.decode("utf-8")
                current_char = read_next_bytes(fid, 1, "c")[0]
            num_points2D = read_next_bytes(fid, num_bytes=8,
                                           format_char_sequence="Q")[0]
            x_y_id_s = read_next_bytes(fid, num_bytes=24*num_points2D,
                                       format_char_sequence="ddq"*num_points2D)
            xys = np.column_stack([tuple(map(float, x_y_id_s[0::3])),
                                   tuple(map(float, x_y_id_s[1::3]))])
            point3D_ids = np.array(tuple(map(int, x_y_id_s[2::3])))
            images[image_id] = Image_(
                id=image_id, qvec=qvec, tvec=tvec,
                camera_id=camera_id, name=image_name,
                xys=xys, point3D_ids=point3D_ids)
    return images


def read_intrinsics_binary(path_to_model_file):
    """
    see: src/base/reconstruction.cc
        void Reconstruction::WriteCamerasBinary(const std::string& path)
        void Reconstruction::ReadCamerasBinary(const std::string& path)
    """
    cameras = {}
    with open(path_to_model_file, "rb") as fid:
        num_cameras = read_next_bytes(fid, 8, "Q")[0]
        for _ in range(num_cameras):
            camera_properties = read_next_bytes(
                fid, num_bytes=24, format_char_sequence="iiQQ")
            camera_id = camera_properties[0]
            model_id = camera_properties[1]
            model_name = CAMERA_MODEL_IDS[camera_properties[1]].model_name
            width = camera_properties[2]
            height = camera_properties[3]
            num_params = CAMERA_MODEL_IDS[model_id].num_params
            params = read_next_bytes(fid, num_bytes=8*num_params,
                                     format_char_sequence="d"*num_params)
            cameras[camera_id] = Camera(id=camera_id,
                                        model=model_name,
                                        width=width,
                                        height=height,
                                        params=np.array(params))
        assert len(cameras) == num_cameras
    return cameras



In [5]:
import numpy as np
import torch
import os
from PIL import Image

# navigate to the dataset folder
base_dir = '/home/archmelow/workfolder/dataset/mipnerf360'

# dataset name
dataset_name = 'bonsai'

base_dir = os.path.join(base_dir, dataset_name)

os.chdir(base_dir)

# images - intrinsic camera binary file, cameras - extrinsic cameras binary file
cameras_extrinsic_file = os.path.join(base_dir, "sparse/0", "images.bin")
cameras_intrinsic_file = os.path.join(base_dir, "sparse/0", "cameras.bin")
images_folder = os.path.join(base_dir, 'images')

cam_extrinsics = read_extrinsics_binary(cameras_extrinsic_file)
cam_intrinsics = read_intrinsics_binary(cameras_intrinsic_file)

print(f'camera extrinsics look like : {cam_extrinsics}')
print(f'camera intrinsics look like : {cam_intrinsics}')

/home/archmelow/anaconda3/envs/gs_tst/lib/python3.7/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


camera extrinsics look like : {1: Image_(id=1, qvec=array([ 0.82376014, -0.22333718, -0.44771114, -0.26663545]), tvec=array([ 0.25063347, -2.21530033,  3.59113953]), camera_id=1, name='DSCF5565.JPG', xys=array([[1430.42085064,  196.91835449],
       [2202.07181053,  224.32566731],
       [2141.12377821,  228.33286306],
       ...,
       [1887.6367235 , 1023.96337949],
       [ 801.5926319 ,  934.962469  ],
       [2388.66803963, 1852.23916894]]), point3D_ids=array([-1, -1, -1, ..., -1, -1, -1])), 2: Image_(id=2, qvec=array([ 0.83221362, -0.20818602, -0.44010108, -0.26531133]), tvec=array([ 0.32420363, -2.17684545,  3.54304157]), camera_id=1, name='DSCF5566.JPG', xys=array([[1610.37665772,   93.87493148],
       [1499.19318101,  161.92525033],
       [1499.82193855,  176.32583855],
       ...,
       [1514.90217895, 1567.21769614],
       [ 841.6562198 ,  901.82796347],
       [2407.18165867, 1850.75416867]]), point3D_ids=array([    -1,     -1,     -1, ..., 129835,     -1,     -1])), 3

In [6]:
import random
import math

# read a camera using the extrinsics and intrinsics

print(f'Number of cameras (viewpoints) : {len(cam_extrinsics)}')
random_cam_idx = random.randint(0, len(cam_extrinsics))

print(f'Read camera {random_cam_idx}..')

extr = cam_extrinsics[random_cam_idx]
intr = cam_intrinsics[extr.camera_id]

image_plane_x = intr.width # image frame height (x) -> in pixels
image_plane_y = intr.height # image frame width (y) -> in pixels
uid = intr.id # Camera ID
R = np.transpose(qvec2rotmat(extr.qvec)) # euler rotation matrix (world coord)
T = np.array(extr.tvec) # translation matrix (world coord)
f_x = intr.params[0]
f_y = intr.params[1]

# assume the camera model to be 'pinhole' model
# field of view (FOV) given the focal lengths and the image plane h,w -> in radian
FovY = 2*math.atan(image_plane_y/(2*f_y))
FovX = 2*math.atan(image_plane_x/(2*f_x))

image_path = os.path.join(images_folder, os.path.basename(extr.name))
image = Image.open(image_path)




Number of cameras (viewpoints) : 292
Read camera 209..


In [7]:
# compact camera info class

attribute_lst = ['uid','R','T','FovY','FovX','image','image_path','width', 'height']

CompactCamera = collections.namedtuple('CompactCamera', attribute_lst)

cur_camera_info = CompactCamera(uid=uid, R=R, T=T, FovY=FovY, FovX=FovX, image=image,
                              image_path=image_path, width=image_plane_x, height=image_plane_y)



In [8]:
    

# load all cameras

print(f'Number of cameras (viewpoints) : {len(cam_extrinsics)}')

cam_infos = []

for cam_idx in range(1, len(cam_extrinsics)+1):
    print(f'Read camera {cam_idx}..')

    extr = cam_extrinsics[cam_idx]
    intr = cam_intrinsics[extr.camera_id]

    image_plane_x = intr.width # image frame height (x) -> in pixels
    image_plane_y = intr.height # image frame width (y) -> in pixels
    uid = intr.id # Camera ID
    R = np.transpose(qvec2rotmat(extr.qvec)) # euler rotation matrix (world coord)
    T = np.array(extr.tvec) # translation matrix (world coord)
    #print(R.shape, T.shape)
    f_x = intr.params[0]
    f_y = intr.params[1]

    # assume the camera model to be 'pinhole' model
    # field of view (FOV) given the focal lengths and the image plane h,w -> in radian
    FovY = 2*math.atan(image_plane_y/(2*f_y))
    FovX = 2*math.atan(image_plane_x/(2*f_x))

    image_path = os.path.join(images_folder, os.path.basename(extr.name))
    image = Image.open(image_path)

    # compact camera info class

    attribute_lst = ['uid','R','T','FovY','FovX','image','image_path','width', 'height']

    CompactCamera = collections.namedtuple('CompactCamera', attribute_lst)

    cur_camera_info = CompactCamera(uid=uid, R=R, T=T, FovY=FovY, FovX=FovX, image=image,
                                image_path=image_path, width=image_plane_x, height=image_plane_y)
    
    cam_infos.append(cur_camera_info)

    


Number of cameras (viewpoints) : 292
Read camera 1..
Read camera 2..
Read camera 3..
Read camera 4..
Read camera 5..
Read camera 6..
Read camera 7..
Read camera 8..
Read camera 9..
Read camera 10..
Read camera 11..
Read camera 12..
Read camera 13..
Read camera 14..
Read camera 15..
Read camera 16..
Read camera 17..
Read camera 18..
Read camera 19..
Read camera 20..
Read camera 21..
Read camera 22..
Read camera 23..
Read camera 24..
Read camera 25..
Read camera 26..
Read camera 27..
Read camera 28..
Read camera 29..
Read camera 30..
Read camera 31..
Read camera 32..
Read camera 33..
Read camera 34..
Read camera 35..
Read camera 36..
Read camera 37..
Read camera 38..
Read camera 39..
Read camera 40..
Read camera 41..
Read camera 42..
Read camera 43..
Read camera 44..
Read camera 45..
Read camera 46..
Read camera 47..
Read camera 48..
Read camera 49..
Read camera 50..
Read camera 51..
Read camera 52..
Read camera 53..
Read camera 54..
Read camera 55..
Read camera 56..
Read camera 57..
Rea

In [9]:
# compute W2C matrix

def getWorld2Camera(R, T):
    w2c = np.zeros((4, 4))
    w2c[:3, :3] = R.transpose()
    w2c[:3, 3] = T
    w2c[3, 3] = 1.0
    return torch.tensor(w2c, dtype=torch.float32)

# compute C2W matrix

def getCamera2World(R, T):
    w2c = np.zeros((4, 4))
    w2c[:3, :3] = R.transpose()
    w2c[:3, 3] = T
    w2c[3, 3] = 1.0
    c2w = np.linalg.inv(w2c)
    return torch.tensor(c2w, dtype=torch.float32)


def ndc_rays(H, W, focal, near, rays_o, rays_d):
    # Shift ray origins to near plane
    t = (near - rays_o[..., 2]) / rays_d[..., 2]
    rays_o = rays_o + t[..., None] * rays_d

    # Projection
    o0 = 1. / (W / (2. * focal)) * rays_o[..., 0] / rays_o[..., 2]
    o1 = 1. / (H / (2. * focal)) * rays_o[..., 1] / rays_o[..., 2]
    o2 = 1. - 2. * near / rays_o[..., 2]

    d0 = 1. / (W / (2. * focal)) * (rays_d[..., 0] / rays_d[..., 2] - rays_o[..., 0] / rays_o[..., 2])
    d1 = 1. / (H / (2. * focal)) * (rays_d[..., 1] / rays_d[..., 2] - rays_o[..., 1] / rays_o[..., 2])
    d2 = 2. * near / rays_o[..., 2]

    rays_o = torch.stack([o0, o1, o2], -1)
    rays_d = torch.stack([d0, d1, d2], -1)

    return rays_o, rays_d

In [10]:
# rays w.r.t one image

rand_cam_idx = random.randint(1, len(cam_extrinsics)-1)
cam = cam_infos[rand_cam_idx]
c2w = getCamera2World(cam.R, cam.T)

W = image_plane_x
H = image_plane_y
image_plane_x = image_plane_x //8
image_plane_y = image_plane_y //8

i, j = torch.meshgrid(torch.linspace(0, image_plane_x-1, image_plane_x),
                      torch.linspace(0, image_plane_y-1, image_plane_y))
i, j = i.T, j.T

focal = [f_x * image_plane_x / W, f_y * image_plane_y / H]

dirs = torch.stack([(i - (image_plane_x/2))/focal[0],
                    -(j - (image_plane_y/2))/focal[1],
                    -torch.ones_like(i)], -1)

print(dirs.shape)

rays_d = torch.sum(dirs[..., np.newaxis, :] * c2w[:3, :3], -1)
rays_o = c2w[:3, -1].expand(rays_d.shape)


                          
rays_d = rays_d.view(-1, 3)[::1000]
rays_o = rays_o.view(-1, 3)[::1000]








torch.Size([259, 389, 3])


/home/archmelow/anaconda3/envs/gs_tst/lib/python3.7/site-packages/torch/functional.py:478: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at  /opt/conda/conda-bld/pytorch_1659484801627/work/aten/src/ATen/native/TensorShape.cpp:2894.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


In [11]:
rays_d.shape

torch.Size([100751, 3])

In [12]:
import plotly.graph_objs as go
import numpy as np
import torch

# Create a list of traces for each ray
traces = []

# Add orange dots at the origin points of the rays
origin_trace = go.Scatter3d(
    x=rays_o[:, 0],
    y=rays_o[:, 1],
    z=rays_o[:, 2],
    mode='markers',
    marker=dict(
        size=4,
        color='orange',  # Set color to orange
        opacity=0.8
    )
)
traces.append(origin_trace)

# Add lines for the rays
for start, end in zip(rays_o, rays_o + rays_d):
    trace = go.Scatter3d(
        x=[start[0], end[0]],
        y=[start[1], end[1]],
        z=[start[2], end[2]],
        mode='lines',
        line=dict(color='blue', width=2)
    )
    traces.append(trace)

# Create a Plotly figure
fig = go.Figure(data=traces)

# Set layout
fig.update_layout(
    scene=dict(
        xaxis=dict(title='X'),
        yaxis=dict(title='Y'),
        zaxis=dict(title='Z'),
    ),
    margin=dict(l=0, r=0, t=0, b=0)
)

# Show the figure
fig.show()

KeyboardInterrupt: 

In [ ]:
# run tensoRF under this setting, and save the weight
print(os.getcwd())




/home/archmelow/workfolder/dataset/mipnerf360/bonsai
